In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/MIP_1_Facility_Location_Allocation/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-18 12:28:30.767080


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

Here is a mathematical formulation of the problem:

Parameters:
- Let $i \in I$ denote the set of possible facility locations.
- Let $j \in J$ denote the set of customers.
- Let $c_{ij}$ be the transportation cost from facility $i$ to customer $j$.
- Let $f_i$ be the fixed annual activation cost for facility $i$.
- Let $M_i$ be the maximum annual service volume at facility $i$.
- Let $d_j$ be the demand of customer $j$.

Decision Variables:
- Let $x_{ij}$ be a binary variable equal to 1 if customer $j$ is serviced by facility $i$, and 0 otherwise.
- Let $y_i$ be a binary variable equal to 1 if facility $i$ is opened, and 0 otherwise.

Objective Function:
The objective is to minimize the overall costs:
\[ \min \sum_{i \in I} \sum_{j \in J} c_{ij}x_{ij} + \sum_{i \in I} f_i y_i \]

Constraints:
1. Each customer's demand must be met:
\[ \sum_{i \in I} x_{ij} = 1 \quad \forall j \in J \]

2. No facility can exceed its maximum service volume:
\[ \sum_{j \in J} d_j x_{ij} \leq M_i y_i \quad \forall i \in I \]

3. Customers can only be serviced by an opened facility:
\[ x_{ij} \leq y_i \quad \forall i \in I, j \in J \]

4. All decision variables are binary:
\[ x_{ij} \in \{0,1\} \quad \forall i \in I, j \in J \]
\[ y_i \in \{0,1\} \quad \forall i \in I \]

This is a mixed-integer linear programming (MILP) formulation of the problem, which can be solved using standard optimization solvers such as CPLEX, Gurobi, or GLPK.

In [6]:
print(response_1_text)

Here is a mathematical formulation of the problem:

Parameters:
- Let $i \in I$ denote the set of possible facility locations.
- Let $j \in J$ denote the set of customers.
- Let $c_{ij}$ be the transportation cost from facility $i$ to customer $j$.
- Let $f_i$ be the fixed annual activation cost for facility $i$.
- Let $M_i$ be the maximum annual service volume at facility $i$.
- Let $d_j$ be the demand of customer $j$.

Decision Variables:
- Let $x_{ij}$ be a binary variable equal to 1 if customer $j$ is serviced by facility $i$, and 0 otherwise.
- Let $y_i$ be a binary variable equal to 1 if facility $i$ is opened, and 0 otherwise.

Objective Function:
The objective is to minimize the overall costs:
\[ \min \sum_{i \in I} \sum_{j \in J} c_{ij}x_{ij} + \sum_{i \in I} f_i y_i \]

Constraints:
1. Each customer's demand must be met:
\[ \sum_{i \in I} x_{ij} = 1 \quad \forall j \in J \]

2. No facility can exceed its maximum service volume:
\[ \sum_{j \in J} d_j x_{ij} \leq M_i y_i \quad 

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

Here is a Python Pyomo code for the optimization problem:

```python
from pyomo.environ import *

# Sample data
I = range(3)  # set of possible facility locations
J = range(5)  # set of customers
c = {(i, j): (i + 1) * (j + 1) for i in I for j in J}  # transportation cost from facility i to customer j
f = {i: (i + 1) * 100 for i in I}  # fixed annual activation cost for facility i
M = {i: (i + 1) * 1000 for i in I}  # maximum annual service volume at facility i
d = {j: (j + 1) * 100 for j in J}  # demand of customer j

# Pyomo model
model = ConcreteModel()

# Sets
model.I = Set(initialize=I)
model.J = Set(initialize=J)

# Parameters
model.c = Param(model.I, model.J, initialize=c)
model.f = Param(model.I, initialize=f)
model.M = Param(model.I, initialize=M)
model.d = Param(model.J, initialize=d)

# Variables
model.x = Var(model.I, model.J, within=Binary)  # 1 if customer j is serviced by facility i, 0 otherwise
model.y = Var(model.I, within=Binary)  # 1 if facility i is opened, 0 otherwise

# Objective Function
def obj_rule(model):
    return sum(model.c[i, j] * model.x[i, j] for i in model.I for j in model.J) + sum(model.f[i] * model.y[i] for i in model.I)
model.obj = Objective(rule=obj_rule, sense=minimize)

# Constraints
def demand_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.I) == 1
model.demand_constraint = Constraint(model.J, rule=demand_constraint_rule)

def max_service_volume_rule(model, i):
    return sum(model.d[j] * model.x[i, j] for j in model.J) <= model.M[i] * model.y[i]
model.max_service_volume = Constraint(model.I, rule=max_service_volume_rule)

def serviced_by_opened_facility_rule(model, i, j):
    return model.x[i, j] <= model.y[i]
model.serviced_by_opened_facility = Constraint(model.I, model.J, rule=serviced_by_opened_facility_rule)

# Solve the model
solver = SolverFactory('glpk')  # Use GLPK solver
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Objective value:", model.obj())
```

This code uses the Pyomo library for modeling and solving the optimization problem. The sample data is defined at the beginning of the code, and you can replace it with your own data. The model is then created, and the objective function and constraints are defined. Finally, the model is solved using the GLPK solver, and the results are printed.

In [10]:
print(response_2_text)

Here is a Python Pyomo code for the optimization problem:

```python
from pyomo.environ import *

# Sample data
I = range(3)  # set of possible facility locations
J = range(5)  # set of customers
c = {(i, j): (i + 1) * (j + 1) for i in I for j in J}  # transportation cost from facility i to customer j
f = {i: (i + 1) * 100 for i in I}  # fixed annual activation cost for facility i
M = {i: (i + 1) * 1000 for i in I}  # maximum annual service volume at facility i
d = {j: (j + 1) * 100 for j in J}  # demand of customer j

# Pyomo model
model = ConcreteModel()

# Sets
model.I = Set(initialize=I)
model.J = Set(initialize=J)

# Parameters
model.c = Param(model.I, model.J, initialize=c)
model.f = Param(model.I, initialize=f)
model.M = Param(model.I, initialize=M)
model.d = Param(model.J, initialize=d)

# Variables
model.x = Var(model.I, model.J, within=Binary)  # 1 if customer j is serviced by facility i, 0 otherwise
model.y = Var(model.I, within=Binary)  # 1 if facility i is opened, 0 otherw

### Code Executability

In [11]:
from pyomo.environ import *

# Sample data
I = range(3)  # set of possible facility locations
J = range(5)  # set of customers
c = {(i, j): (i + 1) * (j + 1) for i in I for j in J}  # transportation cost from facility i to customer j
f = {i: (i + 1) * 100 for i in I}  # fixed annual activation cost for facility i
M = {i: (i + 1) * 1000 for i in I}  # maximum annual service volume at facility i
d = {j: (j + 1) * 100 for j in J}  # demand of customer j

# Pyomo model
model = ConcreteModel()

# Sets
model.I = Set(initialize=I)
model.J = Set(initialize=J)

# Parameters
model.c = Param(model.I, model.J, initialize=c)
model.f = Param(model.I, initialize=f)
model.M = Param(model.I, initialize=M)
model.d = Param(model.J, initialize=d)

# Variables
model.x = Var(model.I, model.J, within=Binary)  # 1 if customer j is serviced by facility i, 0 otherwise
model.y = Var(model.I, within=Binary)  # 1 if facility i is opened, 0 otherwise

# Objective Function
def obj_rule(model):
    return sum(model.c[i, j] * model.x[i, j] for i in model.I for j in model.J) + sum(model.f[i] * model.y[i] for i in model.I)
model.obj = Objective(rule=obj_rule, sense=minimize)

# Constraints
def demand_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.I) == 1
model.demand_constraint = Constraint(model.J, rule=demand_constraint_rule)

def max_service_volume_rule(model, i):
    return sum(model.d[j] * model.x[i, j] for j in model.J) <= model.M[i] * model.y[i]
model.max_service_volume = Constraint(model.I, rule=max_service_volume_rule)

def serviced_by_opened_facility_rule(model, i, j):
    return model.x[i, j] <= model.y[i]
model.serviced_by_opened_facility = Constraint(model.I, model.J, rule=serviced_by_opened_facility_rule)

# Solve the model
solver = SolverFactory('glpk')  # Use GLPK solver
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Objective value:", model.obj())

Status: ok
Objective value: 230.0


### Solution Correctness

In [13]:
from pyomo.environ import *

# Sample data
I = range(1, 4)  # Set of possible locations for opening facilities
J = range(1, 6)  # Set of customers
f = {1: 1000, 2: 1200, 3: 1100}  # Fixed annual activation cost for opening a facility
m = {1: 10000, 2: 12000, 3: 11000}  # Maximum annual service volume at a facility
d = {1: 1000, 2: 2000, 3: 1500, 4: 1800, 5: 2200}  # Annual demand of customers
c = {  # Transportation cost for servicing a customer from a facility
    (1, 1): 50, (1, 2): 60, (1, 3): 55, (1, 4): 70, (1, 5): 80,
    (2, 1): 70, (2, 2): 65, (2, 3): 75, (2, 4): 60, (2, 5): 90,
    (3, 1): 60, (3, 2): 50, (3, 3): 65, (3, 4): 80, (3, 5): 70
}
# Pyomo model
model = ConcreteModel()

# Sets
model.I = Set(initialize=I)
model.J = Set(initialize=J)

# Parameters

# Variables
model.x = Var(model.I, model.J, within=Binary)  # 1 if customer j is serviced by facility i, 0 otherwise
model.y = Var(model.I, within=Binary)  # 1 if facility i is opened, 0 otherwise

# Objective Function
def obj_rule(model):
    return sum(c[i, j] * model.x[i, j] for i in model.I for j in model.J) + sum(f[i] * model.y[i] for i in model.I)
model.obj = Objective(rule=obj_rule, sense=minimize)

# Constraints
def demand_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.I) == 1
model.demand_constraint = Constraint(model.J, rule=demand_constraint_rule)

def max_service_volume_rule(model, i):
    return sum(d[j] * model.x[i, j] for j in model.J) <= m[i] * model.y[i]
model.max_service_volume = Constraint(model.I, rule=max_service_volume_rule)

def serviced_by_opened_facility_rule(model, i, j):
    return model.x[i, j] <= model.y[i]
model.serviced_by_opened_facility = Constraint(model.I, model.J, rule=serviced_by_opened_facility_rule)

# Solve the model
solver = SolverFactory('glpk')  # Use GLPK solver
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Objective value:", model.obj())

Status: ok
Objective value: 1315.0
